In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.data import FragmentIonIntensityDataset

import os

ModuleNotFoundError: No module named 'keras'

In [14]:
model_name = 'f_r_single_both_d5_static_0.0005_aoQ'
model_name = 'b_r_single_both_d5_static_0.0005_99A'
model_name = 'p_r_single_both_d5_static_0.0005_XQO'
model_name = 'b_r_single_both_d5_static_0.0005_5DA'

model = keras.saving.load_model('/nfs/home/students/d.lochert/projects/astral/dlomix-astral/saved_models/%s.keras' % model_name,
                                custom_objects={'masked_spectral_distance': masked_spectral_distance, 'masked_pearson_correlation_distance': masked_pearson_correlation_distance}, 
                                compile=True, safe_mode=True)
model.summary()

Model: "transformer_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            multiple                  6656      
                                                                 
 dense_26 (Dense)            multiple                  3328      
                                                                 
 dense_27 (Dense)            multiple                  131584    
                                                                 
 trans_block_25 (TransBlock  multiple                  524545    
 )                                                               
                                                                 
 trans_block_26 (TransBlock  multiple                  524545    
 )                                                               
                                                                 
 trans_block_27 (TransBlock  multiple          

In [ ]:
int_data = FragmentIonIntensityDataset.load_from_disk("/cmnfs/proj/prosit_astral/datasets/_dlomix_saved/full_dataset")

In [53]:
#test_batch = [m for m in int_data.tensor_train_data.take(42)][0][0]

test_batch = [m for m in int_data.tensor_val_data.take(42)][1]
test_batch

({'modified_sequence': <tf.Tensor: shape=(1024, 30), dtype=int64, numpy=
  array([[11,  6,  1, ...,  0,  0,  0],
         [ 1, 16, 14, ...,  0,  0,  0],
         [ 8, 13, 18, ...,  0,  0,  0],
         ...,
         [ 4, 10, 16, ...,  0,  0,  0],
         [ 4, 18, 17, ...,  0,  0,  0],
         [ 4, 17,  7, ...,  0,  0,  0]])>,
  'charge_oh': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
  array([[0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         ...,
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.]], dtype=float32)>,
  'collision_energy': <tf.Tensor: shape=(1024,), dtype=float32, numpy=
  array([0.2382246, 0.3      , 0.29     , ..., 0.3      , 0.3      ,
         0.27     ], dtype=float32)>,
  'method_nr_oh': <tf.Tensor: shape=(1024, 2), dtype=float32, numpy=
  array([[0., 1.],
         [1., 0.],
         [0., 1.],
         ...,
         [1., 0.],
         [1., 0.],
      

In [5]:
def make_suffled_batch(batch, attribute_name='all'):
    shuffled_batch = batch.copy()

    if attribute_name == 'all':
        shuffled_batch['charge_oh'] = tf.random.shuffle(shuffled_batch['charge_oh'])
        shuffled_batch['collision_energy'] = tf.random.shuffle(shuffled_batch['collision_energy'])
        shuffled_batch['method_nr_oh'] = tf.random.shuffle(shuffled_batch['method_nr_oh'])
        shuffled_batch['machine_oh'] = tf.random.shuffle(shuffled_batch['machine_oh'])

        return shuffled_batch
    
    else:
        shuffled_batch[attribute_name] = tf.random.shuffle(shuffled_batch[attribute_name])
        return shuffled_batch

In [24]:
loss_normal = []
loss_charge = []
loss_collis = []
loss_method = []
loss_machin = []
loss_all = []



test_batches = [m for m in int_data.tensor_val_data.take(5)]

for test_batch in test_batches:

    test_batch_normal = test_batch[0]
    test_batch_charge = make_suffled_batch(test_batch_normal, 'charge_oh')
    test_batch_collis = make_suffled_batch(test_batch_normal, 'collision_energy')
    test_batch_method = make_suffled_batch(test_batch_normal, 'method_nr_oh')
    test_batch_machin = make_suffled_batch(test_batch_normal, 'machine_oh')
    test_batch_all = make_suffled_batch(test_batch_normal, 'all')

    loss_normal.append(tf.reduce_mean( masked_spectral_distance(test_batch[1], model.predict(test_batch_normal)) ))
    loss_charge.append(tf.reduce_mean( masked_spectral_distance(test_batch[1], model.predict(test_batch_charge)) ))
    loss_collis.append(tf.reduce_mean( masked_spectral_distance(test_batch[1], model.predict(test_batch_collis)) ))
    loss_method.append(tf.reduce_mean( masked_spectral_distance(test_batch[1], model.predict(test_batch_method)) ))
    loss_machin.append(tf.reduce_mean( masked_spectral_distance(test_batch[1], model.predict(test_batch_machin)) ))
    loss_all.append(tf.reduce_mean( masked_spectral_distance(test_batch[1], model.predict(test_batch_all)) ))

print(np.mean(loss_normal))
print(np.mean(loss_charge))
print(np.mean(loss_collis))
print(np.mean(loss_method))
print(np.mean(loss_machin))
print(np.mean(loss_all))
    

32/32 [==============================] - 5s 142ms/step
0.17815964
0.3675637
0.21478792
0.32385594
0.40777493
0.54132795


In [65]:
# shuffle metadata
shuffled_batch = test_batch[0].copy()

# shuffled_batch['charge_oh'] = tf.random.shuffle(shuffled_batch['charge_oh'])
# shuffled_batch['collision_energy'] = tf.random.shuffle(shuffled_batch['collision_energy'])
# shuffled_batch['method_nr_oh'] = tf.random.shuffle(shuffled_batch['method_nr_oh'])
shuffled_batch['machine_oh'] = tf.random.shuffle(shuffled_batch['machine_oh'])

shuffled_batch

{'modified_sequence': <tf.Tensor: shape=(1024, 30), dtype=int64, numpy=
 array([[11,  6,  1, ...,  0,  0,  0],
        [ 1, 16, 14, ...,  0,  0,  0],
        [ 8, 13, 18, ...,  0,  0,  0],
        ...,
        [ 4, 10, 16, ...,  0,  0,  0],
        [ 4, 18, 17, ...,  0,  0,  0],
        [ 4, 17,  7, ...,  0,  0,  0]])>,
 'charge_oh': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=float32)>,
 'collision_energy': <tf.Tensor: shape=(1024,), dtype=float32, numpy=
 array([0.2382246, 0.3      , 0.29     , ..., 0.3      , 0.3      ,
        0.27     ], dtype=float32)>,
 'method_nr_oh': <tf.Tensor: shape=(1024, 2), dtype=float32, numpy=
 array([[0., 1.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]], dtype=float32

In [66]:
batch_prediction = model.predict(test_batch[0])
shuffled_batch_prediction = model.predict(shuffled_batch)

print(tf.reduce_mean( masked_spectral_distance(test_batch[1], batch_prediction) ))
print(tf.reduce_mean( masked_spectral_distance(test_batch[1], shuffled_batch_prediction) ))

32/32 [==============================] - 4s 136ms/step
tf.Tensor(0.16962747, shape=(), dtype=float32)
tf.Tensor(0.39736664, shape=(), dtype=float32)


In [15]:
eval_batches = [m for m in int_data.tensor_val_data.take(50)]

pred_ast = []
pred_tof = []
pred_lum = []

true_ast = []
true_tof = []
true_lum = []


for j in range(len(eval_batches)):

    pred = model.predict(eval_batches[j][0])

    batch_size = pred.shape[0]

    mach = np.split(eval_batches[j][0]['machine_oh'], batch_size, axis=0)
    pred = np.split(pred, batch_size, axis=0)
    true = np.split(eval_batches[j][1], batch_size, axis=0)


    for i in range(batch_size):

        if np.argmax(mach[i]) == 0:
            pred_ast.append(pred[i])
            true_ast.append(true[i])
        elif np.argmax(mach[i]) == 1:
            pred_tof.append(pred[i])
            true_tof.append(true[i])
        elif np.argmax(mach[i]) == 2:
            pred_lum.append(pred[i])
            true_lum.append(true[i])

pred_ast = np.concatenate(pred_ast, axis=0)
pred_tof = np.concatenate(pred_tof, axis=0)
pred_lum = np.concatenate(pred_lum, axis=0)

true_ast = np.concatenate(true_ast, axis=0)
true_tof = np.concatenate(true_tof, axis=0)
true_lum = np.concatenate(true_lum, axis=0)

print('AST: ', tf.reduce_mean( masked_spectral_distance(true_ast, pred_ast) ))
print('TOF: ', tf.reduce_mean( masked_spectral_distance(true_tof, pred_tof) ))
print('LUM: ', tf.reduce_mean( masked_spectral_distance(true_lum, pred_lum) ))

32/32 [==============================] - 2s 69ms/step
AST:  tf.Tensor(0.23037277, shape=(), dtype=float32)
TOF:  tf.Tensor(0.16874826, shape=(), dtype=float32)
LUM:  tf.Tensor(0.17807077, shape=(), dtype=float32)


In [15]:
eval_batches = [m for m in int_data.tensor_val_data.take(50)]

pred_cid = []
pred_hcd = []

true_cid = []
true_hcd = []


for j in range(len(eval_batches)):

    pred = model.predict(eval_batches[j][0])

    batch_size = pred.shape[0]

    mach = np.split(eval_batches[j][0]['method_nr_oh'], batch_size, axis=0)
    pred = np.split(pred, batch_size, axis=0)
    true = np.split(eval_batches[j][1], batch_size, axis=0)

    for i in range(batch_size):

        if np.argmax(mach[i]) == 0:
            pred_cid.append(pred[i])
            true_cid.append(true[i])
        elif np.argmax(mach[i]) == 1:
            pred_hcd.append(pred[i])
            true_hcd.append(true[i])

pred_cid = np.concatenate(pred_cid, axis=0)
pred_hcd = np.concatenate(pred_hcd, axis=0)

true_cid = np.concatenate(true_cid, axis=0)
true_hcd = np.concatenate(true_hcd, axis=0)

print('CID: ', tf.reduce_mean( masked_spectral_distance(true_cid, pred_cid) ))
print('HCD: ', tf.reduce_mean( masked_spectral_distance(true_hcd, pred_hcd) ))

32/32 [==============================] - 4s 127ms/step
CID:  tf.Tensor(0.2084779, shape=(), dtype=float32)
HCD:  tf.Tensor(0.15689628, shape=(), dtype=float32)
